# Otters DB (R/RSQLite)

In this notebook we work with an otter colony database.

We

- Copy and clean an existing table
- Update a table and a record

In [218]:
# Libraries
library(tidyverse)
library(RSQLite)
library(DBI)

In [219]:
# Establish a connection
c <- dbConnect(SQLite(), "data/otters_dump.db")

In [220]:
# List available tables
q <- "
SELECT name FROM sqlite_master WHERE type = 'table';
"

tables <- dbGetQuery(c, q) |> pull() |> print()

 [1] "cohort"                       "cohort_legacy"               
 [3] "dim_activity"                 "dim_cleanliness"             
 [5] "dim_diet"                     "dim_friendliness"            
 [7] "dim_fur_density"              "dim_gender"                  
 [9] "dim_observation_type"         "dim_range_status"            
[11] "dim_relationship_type"        "dim_social_status"           
[13] "employers"                    "employment_contracts"        
[15] "health_checks"                "health_checks_legacy"        
[17] "health_observations"          "otter_measurements"          
[19] "otter_preferences_history"    "otter_relationships"         
[21] "preferences_legacy"           "properties_legacy"           
[23] "roles"                        "v_otter_bmi"                 
[25] "v_otter_current_employment"   "v_otter_current_measurements"
[27] "v_otter_current_preferences"  "v_otter_latest_health"       
[29] "cohort_clean"                


``` SQL
-- MySQL
/*
DESCRIBE cohort;
*/
```

In [221]:
q <- "
-- SQLite
PRAGMA table_info(cohort);
"

cols <- dbGetQuery(c, q)
cols

cid,name,type,notnull,dflt_value,pk
<int>,<chr>,<chr>,<int>,<lgl>,<int>
0,otter_id,INTEGER,0,NA,0
1,cute_name,TEXT,0,NA,0
2,birth_date,REAL,0,NA,0
3,social_status_id,INTEGER,0,NA,0
4,range_status_id,INTEGER,0,NA,0


## Construct a new parent table

- Construct a new table
- Clean up the date
- Inspect the otter statuses

In [222]:
# Delete the table if already present
q <- "
DROP TABLE IF EXISTS cohort_clean;
"

dbExecute(c, q)

[1] 0

In [223]:
# Extract the column metadata
cols <- cols |>
    mutate(col = paste(name, type)) |>
    select(col) |>
    pull() |>
    paste(collapse = ",")

In [224]:
# Construct an empty table
q <- paste("
CREATE TABLE cohort_clean (", cols, ");")

dbExecute(c, q)

[1] 0

In [225]:
# Populate the table
q <- "
INSERT INTO cohort_clean
SELECT *
FROM cohort;
"

dbExecute(c, q)

[1] 30

In [226]:
# Check the contents
q <- "
SELECT * FROM cohort_clean
LIMIT 5;
"

dbGetQuery(c, q)


otter_id,cute_name,birth_date,social_status_id,range_status_id
<int>,<chr>,<dbl>,<int>,<int>
1,Whiskers,17633,1,1
2,Pebble,18050,2,1
3,Mochi,18286,2,1
4,Bubbles,17423,3,1
5,Noodle,18933,1,2


In [227]:
# Check the actual dates
as.Date(
    dbGetQuery(c,q)$birth_date,
    origin = "1970-01-01"
)

[1] "2018-04-12" "2019-06-03" "2020-01-25" "2017-09-14" "2021-11-02"

In [228]:
# Rename birth date column
q <- "
ALTER TABLE cohort_clean RENAME COLUMN birth_date TO days_int;
"

dbExecute(c, q)

[1] 0

``` SQL
-- MySQL
/*
ALTER TABLE cohort_clean
ADD COLUMN birth_date DATE;

UPDATE cohort_clean
SET birth_date = DATE_ADD('1970-01-01', INTERVAL days DAY)
WHERE days IS NOT NULL;;
*/
```

In [ ]:
# Add the necessary columns
q <- "
-- SQLite
ALTER TABLE cohort_clean ADD COLUMN days_str TEXT;
"

dbExecute(c, q)

q <- "
ALTER TABLE cohort_clean ADD COLUMN birth_date TEXT;
"

dbExecute(c, q)

[1] 0

In [ ]:
# Convert the number into a date-ready string
q <- "
UPDATE cohort_clean
SET days_str = CAST(days_int AS INTEGER) || ' days';
"

dbExecute(c, q)

[1] 30

In [ ]:
# Construct the date
q <- "
UPDATE cohort_clean
SET birth_date = date('1970-01-01', days_str);
"

dbExecute(c, q)

[1] 30

In [233]:
# Check the contents
q <- "
SELECT birth_date FROM cohort_clean
LIMIT 5;
"

dbGetQuery(c, q)

birth_date
<chr>
2018-04-12
2019-06-03
2020-01-25
2017-09-14
2021-11-02


In [ ]:
# Look at the social and range status
q <- "
SELECT * FROM dim_range_status
LIMIT 5;
"

dbGetQuery(c, q)

range_status_id,status_code,label,is_minor_only
<int>,<chr>,<chr>,<int>
1,FREE,free range,0
2,CARE,in care,1


In [238]:
q <- "
SELECT * FROM dim_social_status
LIMIT 5;
"

dbGetQuery(c, q)

social_status_id,status_code,label
<int>,<chr>,<chr>
1,MR,Mr
2,MS,Ms
3,MRS,Mrs


``` SQL
-- Common table expression
WITH cohort_view AS (
    /* selection within dbGetQuery() */
)

-- Persistent view (reusable across queries)
 CREATE VIEW cohort_enriched AS
    /* selection within dbExecute() */

-- Temporary table (recallable frozen result)
CREATE TEMP TABLE cohort_tmp AS
    /* selection within dbExecute() */
```

In [239]:
q <- "
SELECT 
    c.cute_name AS name, 
    c.birth_date AS birth_date, 
    s.label AS social_status, 
    r.label AS range_status
FROM cohort_clean AS c, dim_social_status AS s, dim_range_status AS r
WHERE
    c.social_status_id = s.social_status_id
    AND c.range_status_id = r.range_status_id
LIMIT 10;
"

dbGetQuery(c, q)

name,birth_date,social_status,range_status
<chr>,<chr>,<chr>,<chr>
Whiskers,2018-04-12,Mr,free range
Pebble,2019-06-03,Ms,free range
Mochi,2020-01-25,Ms,free range
Bubbles,2017-09-14,Mrs,free range
Noodle,2021-11-02,Mr,in care
Pumpkin,2016-10-31,Mrs,free range
Otto,2015-03-18,Mr,free range
Willow,2019-07-09,Ms,free range
Pudding,2022-05-21,Ms,in care


## Add entries to a table and update the parent table

- Add two new rows to a descendant
- Update the parent table with a new value

In [ ]:
# Mrs Bubbles has become a pioneer in a rewilding effort
q <- "
INSERT INTO dim_range_status (range_status_id, status_code, label, is_minor_only)
VALUES 
    (3, 'WILD', 'wild', 0)
    (4, 'INFIRM', 'infirm', 0);
"

dbExecute(c, q)

[1] 1

In [245]:
q <- "
UPDATE cohort_clean
SET range_status_id = 3
WHERE cute_name = 'Bubbles';
"

dbExecute(c, q)

[1] 1

In [ ]:
# Confirm the changed range status
q <- "
SELECT c.cute_name AS name, r.label AS range_status
FROM cohort_clean AS  c, dim_range_status AS  r
WHERE c.cute_name IN ('Bubbles')
AND c.range_status_id = r.range_status_id;
"

dbGetQuery(c, q)

name,range_status
<chr>,<chr>
Bubbles,wild


In [249]:
dbDisconnect(c)